In [108]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [109]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [110]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "originalFileJPG":"",
    "originalFileGrp":[],
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "dest":{
      "rgb":"",
      "ndvi":"",
      "thumb":"",
    },
    "info":{
      "course":"",
      "area":"",
      "desc":"",
      "alt":""
    },
    "label":[],
    "annotation":[]
  }

  

  dslabel['originalFileJPG'] = img_path[3:]
  dslabel['originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [111]:
def getNDVIimg(filesGrp):


  with rasterio.open([ x for x in filesGrp if ('_MS_R' in x) ][0]) as src:
      band_red = src.read(1).astype(float)/65536.

  with rasterio.open([ x for x in filesGrp if ('_MS_NIR' in x) ][0]) as src:
      band_nir = src.read(1).astype(float)/65536.

  np.seterr(divide='ignore', invalid='ignore')

  # Calculate NDVI
  ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))

  ndvi -= ndvi.min() # ensure the minimal value is 0.0
  ndvi /= ndvi.max() # maximum value in image is now 1.0

  cm = plt.cm.get_cmap('RdYlGn')
  ndvi_cm = cm(ndvi)

  ndvi_img = cv2.normalize(ndvi_cm[:,:,:3], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)

  return ndvi_img

In [112]:
def getalignedRGB(filesGrp_, ndvi_img):
  RGBimg = np.asarray(Image.open(filesGrp_[0]))

  # RGBimg = np.asarray(Image.open(filesGrp_[0]).resize((ndvi_img.shape[1],ndvi_img.shape[0])))
  # features0 = FeatureExtraction(RGBimg)
  # features1 = FeatureExtraction(ndvi_img)

  # matches = feature_matching(features0, features1)

  # print('{} number of matched has been found! in {}'.format(len(matches), filesGrp_[0]))

  # if len(matches) < 10:
  #    raise Exception("Not Enough Matches")

  # H, _ = cv2.findHomography( features0.matched_pts, features1.matched_pts, cv2.RANSAC, 5.0)

  H = np.array([[ 5.93209530e-01, -8.35074813e-03, -2.21474434e+02],
       [ 1.32198817e-02,  5.96460559e-01, -2.01042419e+02],
       [ 1.58520522e-06,  1.69424781e-06,  1.00000000e+00]])

  h, w, c = ndvi_img.shape
  RGBimg_tuned = cv2.warpPerspective(RGBimg, H, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

  return RGBimg_tuned 

In [113]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [114]:
course_ids.get('청도')

'MGC002'

In [115]:
# subFolder = glob.glob(os.path.join(img_folder,'*'))
# To_create_folder = [os.path.join(out_folder, list(filter(lambda y: y in x, course_list))[0],os.path.split(x)[-1][:9].replace('_','') ) for x in subFolder]

In [116]:
# To_create_folder

In [117]:
# import itertools

# iterables = [ To_create_folder, folder_type]

# paths_ = [os.path.join(x[0],x[1]) for x in list(itertools.product(*iterables))]

# for path_ in paths_:
#   if not os.path.exists(path_):
#     os.makedirs(path_)

In [118]:
files = glob.glob(os.path.join(img_folder,'**/DJI_*.JPG'), recursive=True)

len(files)

8320

In [119]:
for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  course_id = course_ids.get(course)
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3].split('_')[-1]
  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"
  alt =  info[4].split('_')[-1]

  # print(out_folder,course,target_date,area,desc,'rgb')
  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'rgb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'thumb')

  if not os.path.exists(path_):
    os.makedirs(path_)


In [120]:
filenameOnly = []
for file_ in files:
  filenameOnly.append(os.path.split(file_)[-1])

In [121]:
target_files = [ x for x in files if ('1.5m'in x or '1.5M'in x ) ][100:140] + [ x for x in files if ('0.5m'in x or '0.5M'in x) ][100:120] + [ x for x in files if ('30m' in x or '30M' in x) ][0:20]

len(target_files)

80

In [122]:
path = os.path.normpath(file_)
path.split(os.sep)

['D:',
 'original',
 '2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본',
 '9_레이크4H',
 '2023_0823_청도그레이스_레이크 4H_30m',
 '20230823_1607-1613',
 'DJI_20230823161309_0320_D.JPG']

In [123]:
glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)

['D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_D.JPG',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_G.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_NIR.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_R.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_RE.TIF']

In [124]:
InfoJsonArray = []

for idx, file_ in enumerate(target_files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3].split('_')[-1]
  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"
  alt =  info[4].split('_')[-1]

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)
  # print(filesGrp)
  file_info = getInfo(filesGrp, file_)

  file_info['info']['course'] = course
  file_info['info']['area'] = area
  file_info['info']['alt'] = alt
  file_info['info']['desc'] = desc
  file_info['info']['date'] = target_date


  # try:
  ndvi_img = getNDVIimg(filesGrp)
  rgb_img = getalignedRGB(filesGrp, ndvi_img)
  resized_ndvi = cv2.resize(ndvi_img, dsize=(2592,1944), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_ndvi)
  im.save(save_name)
  file_info['dest']['ndvi'] = save_name[9:]

  resized_rgb = cv2.resize(rgb_img , dsize=(648,486), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'rgb','rgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_rgb)
  im.save(save_name)
  file_info['dest']['rgb'] = save_name[9:]


  thumb_rgb = cv2.resize(rgb_img , dsize=(324,243), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'thumb','thumbrgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(thumb_rgb)
  im.save(save_name)
  file_info['dest']['thumb'] = save_name[9:]

  # except Exception:
  #   pass  # or you could use 'continue'





  InfoJsonArray.append( file_info)

  
  print(save_name)

  print('{} out of {} files Completed'.format(idx, len(target_files)))



  

D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162427_1286447050278_356634336667_161268000.JPG
0 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162429_1286447053333_356634343333_161248000.JPG
1 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162431_1286447061389_356634355000_161240000.JPG
2 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162433_1286447062500_356634368056_161238000.JPG
3 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162435_1286447066667_356634382222_161229000.JPG
4 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162437_1286447070000_356634395278_161268000.JPG
5 out of 80 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162440_1286447073056_356634414722_161249000.JPG

In [125]:
save_name

'D:\\ToAWS\\MGC002\\20230823\\밸리6H\\미기재\\thumb\\thumbrgb20230823104841_1286502136667_356544710278_139723000.JPG'

In [126]:
len(InfoJsonArray)

80

In [127]:
save_name = os.path.join(out_folder,course_id,target_date, 'data.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)

In [128]:
with open('mydata.json', "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)